In [ ]:
### Install dependencies
!pip install datasets
!pip install spacy
!python -m spacy download pt_core_news_sm
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet
import os
os.kill(os.getpid(), 9)

In [ ]:
import json
import re
import spacy
from openai import AzureOpenAI
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset

# Configuração do cliente base
client = AzureOpenAI(
  azure_endpoint="<azure_endpoint>",
  api_key="<api_key>",
  api_version="2024-02-01"
)

In [ ]:
def sliding_window(text, window_size=400, overlap=50):
    """
    Function that splits a text into sliding windows of size `window_size` with an `overlap`.

    Args:
        text (str): The text to be processed.
        window_size (int): The maximum number of tokens per window.
        overlap (int): The number of overlapping tokens between windows.

    Returns:
        List of str: List of texts divided into sliding windows.
    """
    # Load SpaCy model for Portuguese (or another language, if necessary)
    nlp = spacy.load('pt_core_news_sm')

    # Process the entire text with SpaCy
    doc = nlp(text)

    # Extract tokens
    tokens = [token.text for token in doc]

    # List to store text windows
    windows = []

    # Sliding window implementation
    for i in range(0, len(tokens), window_size - overlap):
        # Capture a window of tokens
        window = tokens[i:i + window_size]
        windows.append(" ".join(window))

        # Stop if we are at the end of the text
        if i + window_size >= len(tokens):
            break

    return windows

import re

def extract_tagged_words(text):
    """
    Extract labeled words from text based on XML-like tags.

    Args:
        text (str): The input text containing tags.

    Returns:
        list: A list of dictionaries with extracted words, their category, subcategory,
              and character positions in the original text.
    """
    dict_categories = {"AGE": "AGE", "PHONE": "CONTACT", "FAX": "CONTACT", "EMAIL": "CONTACT", "URL": "CONTACT",
                   "IP_ADDRESS": "CONTACT", "DATE": "DATE", "IDNUM": "ID", "MEDICAL_RECORD": "ID", "DEVICE": "ID",
                   "HEALTH_PLAN": "ID", "BIOID": "ID", "STREET": "LOCATION", "CITY": "LOCATION", "ZIP": "LOCATION",
                   "STATE": "LOCATION", "COUNTRY": "LOCATION", "LOCATION_OTHER": "LOCATION", "ORGANIZATION": "LOCATION",
                   "HOSPITAL": "LOCATION", "PATIENT": "NAME", "DOCTOR": "NAME", "USERNAME": "NAME", "PROFESSION": "PROFESSION",
                   "OTHER": "OTHER", "LOCATION": "LOCATION"}

    pattern = r"<(.*?)>(.*?)</\1/>"  # Regex to capture tags and content between them
    matches = re.finditer(pattern, text)

    result = []
    for match in matches:
        tag = match.group(1)
        word = match.group(2)
        first_position = match.start(2)  # Start position of the extracted word
        last_position = match.end(2)     # End position of the extracted word
        category = dict_categories.get(tag, "UNKNOWN")  # Main category
        subcategory = tag

        result.append({
            "word": word,
            "category": category,
            "subcategory": subcategory,
            "first_position": first_position,
            "last_position": last_position
        })

    return result

def find_missing_words(predicted_words, labels):
    """
    Finds words present in `labels` that are not in `predicted_words`.

    Args:
        predicted_words (list): List of predicted words.
        labels (list): List of correct words (labels).

    Returns:
        tuple: Number of missing words and a list of those words.
    """
    missing_words = [word for word in labels if word not in predicted_words]
    return len(missing_words), missing_words

def calculate_f1_score(tp, fp, fn, verbose= False):
    """
    Calculate F1 Score, Recall, and Precision.

    Args:
        tp (int): True Positives.
        fp (int): False Positives.
        fn (int): False Negatives.
        verbose (bool): Whether to print detailed metrics.

    Returns:
        tuple: (f1_score, recall, precision)
    """
    # Calculate precision (Total correct divided by number of attempts)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    # Calculate recall (Total correct divided by number of words that should have been masked)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    if verbose:
        print('Recall:', recall)
        print('Precision:', precision)
        print('F1 Score:', f1_score, '\n')

    return f1_score, recall, precision

def eval(extractive_pred, dict_labels, verbose=False):
    """
    Evaluate extractive predictions against labeled data.

    Args:
        extractive_pred (dict): Dictionary containing predicted words and categories.
        dict_labels (dict): Dictionary of ground-truth labels for words.
        verbose (bool): Whether to print detailed evaluation results.

    Returns:
        tuple: F1, Recall, Precision (per word and overall)
    """
    TP, FP, FN = 0, 0, 0
    correct_predicted_words = []
    wrong_predicted_words = []
    predicted_words = []
    wrong_predicted_category = []
    for pred in extractive_pred['preds']:

        # predicted_words.append(pred['word'])
        if pred['word'] in list(dict_labels.keys()):

            if pred['subcategory'] == dict_labels[pred['word']]:
                TP+=1
                correct_predicted_words.append((pred['word'], pred['subcategory']))
                predicted_words.append(pred['word'])
            else:
                FP+=1
                wrong_predicted_category.append((pred['word'], pred['subcategory']))
        else:
            FP+=1
            wrong_predicted_words.append(pred['word'])

    # Calculate False Negatives
    FN, missing_words = find_missing_words(predicted_words, list(dict_labels.keys()))
    if verbose:
        print('Missing words:', missing_words)
        print('Correct Predicted words:', correct_predicted_words)
        print('Correct word but wrong category:', wrong_predicted_category)
        print('Wrong Predicted words:', wrong_predicted_words)
        print('Labels:', dict_labels)

    # Calculate F1 Score
    f1, recall, precision = calculate_f1_score(TP, FP, FN, verbose=verbose)

    return f1, recall, precision

def create_generative_format(text):
    """
    Convert text with tagged entities into a simplified generative format.

    This function replaces each complete tag (opening and closing with content)
    with only its opening tag, discarding the enclosed content.
    It is useful when preparing data for generative anonymization tasks
    where only the entity type needs to be indicated.

    Args:
        text (str): Input text containing XML-like tags with content.

    Returns:
        str: Text transformed into generative format with only opening tags.
    """
    # Regex to capture the opening tag and the content between tags
    pattern = r"<(.*?)>(.*?)</\1/>"

    # Replacement function that uses only the opening tag
    def replace_match(match):
        tag = match.group(1)
        # Returns only the opening tag
        return f"<{tag}>"

    # Replace all occurrences in the text
    replaced_text = re.sub(pattern, replace_match, text)
    return replaced_text

def fix_tags_with_replace(text):
    """
    Fixes malformed tags in the text using replace for each possible case.

    Args:
        text (str): The text generated by the model with malformed tags.

    Returns:
        str: Text with corrected tags.
    """
    # List of tags that need to be fixed
    tags = [
        "AGE", "PHONE", "FAX", "EMAIL", "URL", "IP_ADDRESS", "DATE", "IDNUM",
        "MEDICAL_RECORD", "DEVICE", "HEALTH_PLAN", "BIOID", "STREET", "CITY",
        "ZIP", "STATE", "COUNTRY", "LOCATION_OTHER", "ORGANIZATION", "HOSPITAL",
        "PATIENT", "DOCTOR", "USERNAME", "PROFESSION", "OTHER", "LOCATION"
    ]

    for tag in tags:
        # Fix spaces around the opening tag
        text = text.replace(f"< {tag} >", f"<{tag}>").replace(f"< {tag}>", f"<{tag}>").replace(f"<{tag} >", f"<{tag}>")
        # Fix spaces around the closing tag
        text = text.replace(f"</ {tag} >", f"</{tag}/>").replace(f"</ {tag}>", f"</{tag}/>").replace(f"</{tag} >", f"</{tag}/>")
        # Fix malformed closings with extra slashes
        text = text.replace(f"<{tag}/> ", f"</{tag}/>").replace(f"<{tag}/ >", f"</{tag}/>").replace(f"</{tag}/ >", f"</{tag}/>")
        # Remove spaces between tags and inner content
        text = text.replace(f"<{tag}> ", f"<{tag}>").replace(f" </{tag}>", f"</{tag}/>").replace(f"</{tag}>", f"</{tag}/>")

    return text

def split_tags_with_multiple_words(text):
    """
    Split multi-word entity tags into individual single-word tags.

    This function identifies tags in the format <TAG>content</TAG/>,
    and if the content contains multiple words, it separates them so
    that each word is wrapped in its own <TAG>...</TAG/>.
    If the tag content is already a single word, it remains unchanged.

    Args:
        text (str): Input text containing XML-like tags with entity labels.

    Returns:
        str: Transformed text where multi-word tags are split into single-word tags.
    """
    # Regex to find tags in the format <TAG>content</TAG/>
    pattern = r"<(.*?)>(.*?)</\1/>"

    # Function to replace tags with multiple words
    def split_tag(match):
        tag = match.group(1)  # Tag name
        content = match.group(2)  # Content between tags

        # Check if the content has multiple words
        words = content.split()
        if len(words) > 1:
            # Create a new tag for each word
            return " ".join(f"<{tag}>{word}</{tag}/>" for word in words)
        return match.group(0)  # Return the original tag if it’s a single word

    # Apply regex and replace
    return re.sub(pattern, split_tag, text)

def run_prediction(input_text, model="gpt4-o-mini"):
    try:
        messages = [
            {"role": "system", "content": """
             Você é uma assistente que auxilia na identificação de entidades em textos médicos incluindo tags de marcação no texto original.
             Você deve identificar somente as seguintes entidades, não crie novas entidades além das listadas aqui: "AGE", "PHONE", "EMAIL", "DATE", "IDNUM",
             "MEDICAL_RECORD", "HEALTH_PLAN", "STREET", "CITY","ZIP", "STATE", "COUNTRY", "LOCATION_OTHER", "ORGANIZATION",
             "HOSPITAL", "PATIENT", "DOCTOR", "USERNAME", "PROFESSION", "OTHER. As tags possuem o seguinte formato: <PATIENT> </PATIENT/>
             <DOCTOR> </DOCTOR/> <DATE> </DATE/>. Cada par de tags deve conter apenas uma palavra como mostram os exemplos abaixo."""},

            {"role": "user", "content": """Subjetivo\n # REVISÃO CLÍNICA - GASTROENTEROLOGIA #\n
              #De acordo com o Dr. Roberto Freitas, após endoscopia diagnóstica:\n
              - Suspender o uso de AINES temporariamente devido a risco de gastrite medicamentosa.\n
              - Iniciar omeprazol 40 mg diariamente por 30 dias.\n
              - Introduzir dieta branda, evitando alimentos condimentados e café.\n
              - Agendar retorno em 2 semanas para reavaliação dos sintomas.\n
              - Paciente Romildo Pena, 45 anos, engenheiro, instruído a procurar o serviço de emergência caso apresente vômitos ou dor intensa.\n
              Elaborado e assinado por médico residente Dr. Gabriel Teixeira, telefone (11) 3546-8964, CRM 634278 em 15/09/2022 09:50, consultório Rua da Saúde, 234, São Paulo."""},

            {"role": "assistant", "content": """Subjetivo\n # REVISÃO CLÍNICA - GASTROENTEROLOGIA #\n
              #De acordo com o Dr. <DOCTOR>Roberto</DOCTOR/> <DOCTOR>Freitas</DOCTOR/>, após endoscopia diagnóstica:\n
              - Suspender o uso de AINES temporariamente devido a risco de gastrite medicamentosa. \n
              - Iniciar omeprazol 40 mg diariamente por 30 dias. \n
              - Introduzir dieta branda, evitando alimentos condimentados e café.\n
              - Agendar retorno em 2 semanas para reavaliação dos sintomas.\n
              - Paciente <PATIENT>Romildo</PATIENT/> <PATIENT>Pena</PATIENT/>, <AGE>45</AGE/> anos, <PROFESSION>engenheiro</PROFESSION/>, instruído a procurar o serviço de emergência caso apresente vômitos ou dor intensa.\n
              Elaborado e assinado por médico residente Dr. <DOCTOR>Gabriel</DOCTOR/> <DOCTOR>Teixeira</DOCTOR/>, telefone <PHONE>(11)</PHONE/> <PHONE>3546-8964</PHONE/>, CRM <IDNUM>634278</IDNUM/> em <DATE>15</DATE/>/<DATE>09</DATE/>/<DATE>2022</DATE/> 09:50, consultório <STREET>Rua</STREET/> <STREET>da</STREET/> <STREET>Saúde</STREET/>, 234, <CITY>São</CITY/> <CITY>Paulo</CITY/>.
              """},

            {"role":"user", "content": """Subjetivo
            ### EVOLUCAO MÉDICA - GERIATRIA #
            # ID: Mauro José, 77 anos, natural de Paratinga-BA, procedente de Campinas-SP, casado, 4 filhos, escolaridade 4 anos, católico,
            aposentado (açougueiro), mora com a esposa
            #DIH: 28/08
            #QP:
            Piora de confusão mental e perda de força nas pernas
            #HPMA
            Equipe da geriatria é chamada para avaliar paciente em leito de UER.
            Paciente acompanha em inúmeras especialidades em nossos serviços. A esposa e a filha contam que no último mês o paciente
            começou a apresentar progressão do quadro de demência, associado a paresia de membros inferiores e de mãos, parestesia de mãos,
            enrijecimento de articulações e dores articulares. A esposa refere que antes o paciente apresentava dificuldade para deambular, mas
            conseguia deambular com ajuda, porém desde a última vez que retirou líquor (31/07) acabou perdendo força em MMII para deambular.
            Associada à piora do quadro cognitivo e motor, o paciente apresentou perda do controle de esfíncteres, algo que antes era preservado.
            A esposa disse que o paciente queixa-se de cefaleia intensa diariamente e mantém alucinações visuais e verbais. Esposa refere
            evacuações 3x/dia, com fezes amolecidas. Urina de odor fétido e aspecto avermelhado, associado a disúria e redução do volume
            urinário.
            Em 2007 o paciente teve um quadro parecido sendo internado com insuficiência adrenal.
            #AP:
            1) Síndrome demencial moderada a grave - etiologia não esclarecida (HPN? Lewy?)
            2) Síndrome poliglandular endócrina (ALTO ACTH/ BAIXO CORTISOL)
            * insuficiência adrenal primária
            * hipotireoidismo + gastrite atrófica
            3) HAS / DLP
            4) DRC
            5) HBP
            #Evolução Médica:
            Paciente em leito de enfermaria geral, acompanhado pela filha, alerta, comunicativo e colaborativo, calmo, atenção preservada. Mantem
            boa aceitação da dieta via oral. Baixo debito urinario nas ultimas 24hs (SVD 125ml), apresentando mais 200ml durante a visita pela
            manhã; evacuação ausente. Segue hemodinamicamente estavel, sem intercorrencias no periodo, controles dentro da normalidade, com
            melhora do edema. Em programação de nova sessão de hemodialise hoje a tarde.
            #EXAME FISICO:
            REG, descorado 2+/4 em mucosas , mucosas pouco secas, anictérico, afebril , CAM -
            AR: MV+ sem RA, sat 96% em aa
            ACV: BRNF 2T, extremidades quentes , FC 81 bpm, PA 130x80mmHg
            #EXAMES COMPLEMENTARES:
            (29/08/23): TC cranio: ventriculos aumentados bilateralmente , atrofia global com predominancia me lobos temporais, MTA 4
            bilateralmente - impressão da nossa equipe aguardando laudo oficial
            (01/09/23) USG Rins e Vias Urinarias: Sinais de nefropatia parenquimatoa crônica bilateral;
            #CULTURAS:
            (04/09/23) URC Negativo
            (29/08/23) Uroc: Microbiota diversa (Contaminação)
            (01/09/23) Hemocultura: aguarda resultado
            Impressão
            #HDs
            1) DRC agudizada por desidratação?
            > Realizada primeira sessão de diálise em 02/09 . HD: 02/09; 04/09; 05/09; 07/09
            2) Hipercalemia corrigida
            > sec a DRC agudizada
            3) Delirium em melhora
            > Progressão do quadro neurológico de base? Sec a agudização renal ?
            # Impressão:
            Paciente no momento com sinais vitais estáveis porém com piora da função renal no decorrer da internação. Equipe de nefrologia
            segue em acompanhamento conjunto > realizada primeira dialise dia 02/09. Em programação de nova sessão de dialise hoje;
            Conduta
            #CDs
            - Checo exames - piora das escórias nitrogenadas - em programação de HD hoje pela nefrologia;
            - Checo USG Rins e Vias Urinarias: Sinais de nefropatia parenquimatoa crônica bilateral;
            - Aguarda eletroforese de proteinas;
            - Vigilancia de plaquetopenia (uso heparina 12/12h + uso de heparina durante dialise ??)
            - Reavaliaremos plano terapêutico diariamente, de acordo com resposta ao tratamento instituído e considerando morbidades/
            funcionalidade do paciente.
            - Exames laboratorias diario e seguimento conjunto com a nefrologia.
            - Forneço boletim medico para filha, esclareço duvidas, atualizo plano terapeutico, acolho.
            RODOLFO R3 GERIATRIA + DR ANDRÉ FATTORI.
            Elaborado e assinado por Dr. Rodolfo Antonio De Oliveira Nogueira, Crm 210538 em 07/09/2023 15:32
            """},

             {"role":"assistant", "content": """ Subjetivo
            ### EVOLUCAO MÉDICA - GERIATRIA #
            # ID: <PATIENT>Mauro</PATIENT/> <PATIENT>José</PATIENT/>, <AGE>77</AGE/> anos, natural de <CITY>Paratinga</CITY/>-<STATE>BA</STATE/>, procedente de <CITY>Campinas</CITY/>-<STATE>SP</STATE/>, <OTHER>casado</OTHER/>, 4 filhos, escolaridade 4 anos, <OTHER>católico</OTHER/>,
            <PROFESSION>aposentado</PROFESSION/> (<PROFESSION>açougueiro</PROFESSION/>), mora com a esposa
            #DIH: <DATE>28</DATE/>/<DATE>08</DATE/>
            #QP:
            Piora de confusão mental e perda de força nas pernas
            #HPMA
            Equipe da geriatria é chamada para avaliar paciente em leito de <LOCATION_OTHER>UER</LOCATION_OTHER/>.
            Paciente acompanha em inúmeras especialidades em nossos serviços. A esposa e a filha contam que no último mês o paciente
            começou a apresentar progressão do quadro de demência, associado a paresia de membros inferiores e de mãos, parestesia de mãos,
            enrijecimento de articulações e dores articulares. A esposa refere que antes o paciente apresentava dificuldade para deambular, mas
            conseguia deambular com ajuda, porém desde a última vez que retirou líquor (<DATE>31</DATE/>/<DATE>07</DATE/>) acabou perdendo força em MMII para deambular.
            Associada à piora do quadro cognitivo e motor, o paciente apresentou perda do controle de esfíncteres, algo que antes era preservado.
            A esposa disse que o paciente queixa-se de cefaleia intensa diariamente e mantém alucinações visuais e verbais. Esposa refere
            evacuações 3x/dia, com fezes amolecidas. Urina de odor fétido e aspecto avermelhado, associado a disúria e redução do volume
            urinário.
            Em <DATE>2007</DATE/> o paciente teve um quadro parecido sendo internado com insuficiência adrenal.
            #AP:
            1) Síndrome demencial moderada a grave - etiologia não esclarecida (HPN? Lewy?)
            2) Síndrome poliglandular endócrina (ALTO ACTH/ BAIXO CORTISOL)
            * insuficiência adrenal primária
            * hipotireoidismo + gastrite atrófica
            3) HAS / DLP
            4) DRC
            5) HBP
            #Evolução Médica:
            Paciente em leito de enfermaria geral, acompanhado pela filha, alerta, comunicativo e colaborativo, calmo, atenção preservada. Mantem
            boa aceitação da dieta via oral. Baixo debito urinario nas ultimas 24hs (SVD 125ml), apresentando mais 200ml durante a visita pela
            manhã; evacuação ausente. Segue hemodinamicamente estavel, sem intercorrencias no periodo, controles dentro da normalidade, com
            melhora do edema. Em programação de nova sessão de hemodialise hoje a tarde.
            #EXAME FISICO:
            REG, descorado 2+/4 em mucosas , mucosas pouco secas, anictérico, afebril , CAM -
            AR: MV+ sem RA, sat 96% em aa
            ACV: BRNF 2T, extremidades quentes , FC 81 bpm, PA 130x80mmHg
            #EXAMES COMPLEMENTARES:
            (<DATE>29</DATE/>/<DATE>08</DATE/>/<DATE>23</DATE/>): TC cranio: ventriculos aumentados bilateralmente , atrofia global com predominancia me lobos temporais, MTA 4
            bilateralmente - impressão da nossa equipe aguardando laudo oficial
            (<DATE>01</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>) USG Rins e Vias Urinarias: Sinais de nefropatia parenquimatoa crônica bilateral;
            #CULTURAS:
            (<DATE>04</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>) URC Negativo
            (<DATE>29</DATE/>/<DATE>08</DATE/>/<DATE>23</DATE/>) Uroc: Microbiota diversa (Contaminação)
            (<DATE>01</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>) Hemocultura: aguarda resultado
            Impressão
            #HDs
            1) DRC agudizada por desidratação?
            > Realizada primeira sessão de diálise em <DATE>02</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>. HD: <DATE>02</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>; <DATE>04</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>; <DATE>05</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>; <DATE>07</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>
            2) Hipercalemia corrigida
            > sec a DRC agudizada
            3) Delirium em melhora
            > Progressão do quadro neurológico de base? Sec a agudização renal ?
            # Impressão:
            Paciente no momento com sinais vitais estáveis porém com piora da função renal no decorrer da internação. Equipe de nefrologia
            segue em acompanhamento conjunto > realizada primeira dialise dia <DATE>02</DATE/>/<DATE>09</DATE/>/<DATE>23</DATE/>. Em programação de nova sessão de dialise hoje;
            Conduta
            #CDs
            - Checo exames - piora das escórias nitrogenadas - em programação de HD hoje pela nefrologia;
            - Checo USG Rins e Vias Urinarias: Sinais de nefropatia parenquimatoa crônica bilateral;
            - Aguarda eletroforese de proteinas;
            - Vigilancia de plaquetopenia (uso heparina 12/12h + uso de heparina durante dialise ??)
            - Reavaliaremos plano terapêutico diariamente, de acordo com resposta ao tratamento instituído e considerando morbidades/
            funcionalidade do paciente.
            - Exames laboratorias diario e seguimento conjunto com a nefrologia.
            - Forneço boletim medico para filha, esclareço duvidas, atualizo plano terapeutico, acolho.
            <DOCTOR>RODOLFO</DOCTOR/> R3 GERIATRIA + DR <DOCTOR>ANDRÉ</DOCTOR/> <DOCTOR>FATTORI</DOCTOR/>.
            Elaborado e assinado por Dr. <DOCTOR>Antonio</DOCTOR/> <DOCTOR>De</DOCTOR/> <DOCTOR>Oliveira</DOCTOR/> <DOCTOR>Nogueira</DOCTOR/>, Crm <IDNUM>210538</IDNUM/> em <DATE>07</DATE/>/<DATE>09</DATE/>/<DATE>2023</DATE/> 15:32
            """},

            {"role": "user", "content": """ Subjetivo
              # EVOLUÇÃO ENFERMARIA DE CARDIOLOGIA #
              # ID: Antonio Alves de Paula, 58 anos
              HC: 12735360
              Acompanhado por Maria Ferreira de Paula
              #HD:
              1. Taquicardia ventricular em EE de 24/08/2023
              História de síncopes ambulatorialmente em paciente com miocardiopatia chagásica
              2. Hipotireoidismo induzido por tapazol
              #AP
              Miocardiopatia chagásica
              Hipertireoidismo em uso de tapazol --> evolução para hipotireoidismo
              ICFER
              #HPMA: Paciente com miocardiopatia chagásica acompanha no ambulatório de cardiologia da Unicamp.
              # CONDUTAS:
              - Aguarda chegada do CDI
              Tulio I1 + Arthur R4 + Dra. Adriana
              Elaborado e assinado por Acadêmico Tulio Carmona Moura em 13/09/2023 22:57
              """},

            {"role": "assistant", "content": """Subjetivo
              # EVOLUÇÃO ENFERMARIA DE CARDIOLOGIA #
              # ID: <PATIENT>Antonio</PATIENT/> <PATIENT>Alves</PATIENT/> <PATIENT>de</PATIENT/> <PATIENT>Paula</PATIENT/>, <AGE>58</AGE/> anos
              HC: <MEDICAL_RECORD>12735360</MEDICAL_RECORD/>
              Acompanhado por <PATIENT>Maria</PATIENT/> <PATIENT>Ferreira</PATIENT/> <PATIENT>de</PATIENT/> <PATIENT>Paula</PATIENT/>
              #HD:
              1. Taquicardia ventricular em EE de <DATE>24</DATE/>/<DATE>08</DATE/>/<DATE>2023</DATE/>
              História de síncopes ambulatorialmente em paciente com miocardiopatia chagásica
              2. Hipotireoidismo induzido por tapazol
              #AP
              Miocardiopatia chagásica
              Hipertireoidismo em uso de tapazol --> evolução para hipotireoidismo
              ICFER
              #HPMA: Paciente com miocardiopatia chagásica acompanha no ambulatório de cardiologia da <ORGANIZATION>Unicamp</ORGANIZATION/>.
              # CONDUTAS:
              - Aguarda chegada do CDI
              <DOCTOR>Tulio</DOCTOR/> I1 + <DOCTOR>Arthur</DOCTOR/> R4 + Dra. <DOCTOR>Adriana</DOCTOR/>
              Elaborado e assinado por Acadêmico <DOCTOR>Tulio</DOCTOR/> <DOCTOR>Carmona</DOCTOR/> <DOCTOR>Moura</DOCTOR/> em <DATE>13</DATE/>/<DATE>09</DATE/>/<DATE>2023</DATE/> 22:57"""},


            {"role": "user", "content": input_text}

        ]
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            temperature = 0.0,
            max_tokens = 4096,
            top_p = 1.0,
            frequency_penalty = 0,
            presence_penalty = 0,
            stop = None
        )
    except Exception as e:
        print(str(e))

    return response

In [ ]:
# Load test set
dataset_ = load_dataset("Venturus/AnonyMED-BR")
test_set = dataset_['test']

list_f1 = []
list_recall = []
list_precision = []
list_pred_gen = []
list_syn = []
list_preds = []
list_id = []
for test_sample in tqdm(test_set):

    # Create the sliding window to input on the NLP model
    clean_text = re.sub(r"<[^>]+>", "", test_sample['text'])

    try:
        # Run inference on all windows and post-process data
        windows = sliding_window((clean_text), window_size=1700, overlap=0)
        preds = [run_prediction(window ,model="<path_to_model>").choices[0].message.content for window in windows]
        final_pred = ' '.join(preds)
        fixed_final_pred = fix_tags_with_replace(final_pred).replace(' , ',', ')
        fixed_final_pred = split_tags_with_multiple_words(fixed_final_pred)

        #### Fix some tags if needed
        fixed_final_pred = fixed_final_pred.replace("</DATE/>/<DATE>","/")
        fixed_final_pred = fixed_final_pred.replace("</PHONE/>-<PHONE>","-")

        ### Save prediction in a list
        list_preds.append(fixed_final_pred)

        ### Extractive Format Evaluation ###
        tags = extract_tagged_words(fixed_final_pred)
        extractive_pred = {'id': test_sample['id'], 'preds': tags}

        # Get labels in the evaluation format
        dict_labels = {item['word']: item['subcategory'] for item in test_sample['labels']}

        # Run evaluation
        f1, recall, precision = eval(extractive_pred, dict_labels, verbose=False)

        # Save results
        list_f1.append(f1)
        list_recall.append(recall)
        list_precision.append(precision)

        # Save if it is synthetic or not
        list_syn.append(test_sample['synthetic'])

        # Save example id
        list_id.append(test_sample['id'])

        ### Generative Format Evaluation ###
        list_pred_gen.append({'text': re.sub(r"<[^>]+>", "", test_sample['text']), 'masked_text':create_generative_format(test_sample['text']), 'prediction':create_generative_format(fixed_final_pred)})

    except:
        print('An error occured during model generation step', preds)


### Create a dataframe with the evaluations
save_df = pd.DataFrame()
save_df['id'] = list_id
save_df['Recall'] = list_recall
save_df['Precision'] = list_precision
save_df['F1'] = list_f1
save_df['synthetic'] = list_syn
save_df['Prediction'] = list_preds
save_df.to_csv('GPT_results.csv', index=False)

avg_f1 = sum(list_f1) / len(list_f1) if list_f1 else 0
avg_recall = sum(list_recall) / len(list_recall) if list_recall else 0
avg_precision = sum(list_precision) / len(list_precision) if list_precision else 0

print('\n --------------------------------------------------------------- \n')
print('Recall:', avg_recall)
print('Precision:', avg_precision)
print('F1:', avg_f1)

## Save predictions on the generative format
with open('gpt_generative_predictions.json', 'w', encoding='utf-8') as f:
      json.dump(list_pred_gen, f, ensure_ascii=False, indent=4)

### Evaluation per Entity

In [ ]:
list_entities = ["PHONE", "AGE", "FAX", "EMAIL", "URL", "IP_ADDRESS", "DATE", "IDNUM",
        "MEDICAL_RECORD", "DEVICE", "HEALTH_PLAN", "BIOID", "STREET", "CITY",
        "ZIP", "STATE", "COUNTRY", "LOCATION_OTHER", "ORGANIZATION", "HOSPITAL",
        "PATIENT", "DOCTOR", "USERNAME", "PROFESSION", "OTHER", "LOCATION"]

def eval_entity(extractive_pred, dict_labels, verbose=False):
    """
    Evaluate the performance of extractive predictions against reference labels for entities.

    This function compares predicted words and their subcategories to a reference dictionary of labels.
    It calculates True Positives (TP), False Positives (FP), and False Negatives (FN), and returns
    the corresponding F1 score, recall, and precision. Optionally, it can print detailed information
    about correct and incorrect predictions.

    Args:
        extractive_pred (dict): Dictionary containing predicted entities under the key 'preds',
                                where each prediction is a dictionary with 'word' and 'subcategory'.
        dict_labels (dict): Dictionary of reference labels with words as keys and subcategories as values.
        verbose (bool, optional): If True, prints detailed information about missing words,
                                  correct predictions, and incorrect predictions. Defaults to False.

    Returns:
        tuple: A tuple containing:
            - f1 (float): F1 score for entity predictions.
            - recall (float): Recall score for entity predictions.
            - precision (float): Precision score for entity predictions.
    """
    TP, FP, FN = 0, 0, 0
    correct_predicted_words = []
    wrong_predicted_words = []
    predicted_words = []
    wrong_predicted_category = []
    for pred in extractive_pred['preds']:

        if pred['word'] in list(dict_labels.keys()):

            if pred['subcategory'] == dict_labels[pred['word']]:
                TP+=1
                correct_predicted_words.append((pred['word'], pred['subcategory']))
                predicted_words.append(pred['word'])
            else:
                FP+=1
                wrong_predicted_category.append((pred['word'], pred['subcategory']))
        else:
            FP+=1
            wrong_predicted_words.append(pred['word'])

    # Calculate False Negatives
    FN, missing_words = find_missing_words(predicted_words, list(dict_labels.keys()))
    if verbose:
        print('Missing words:', missing_words)
        print('Correct Predicted words:', correct_predicted_words)
        print('Correct word but wrong category:', wrong_predicted_category)
        print('Wrong Predicted words:', wrong_predicted_words)
        print('Labels:', dict_labels)

    # Calculate F1 Score
    f1, recall, precision = calculate_f1_score(TP, FP, FN, verbose=verbose)

    return f1, recall, precision

In [ ]:
def intersect_by_id(list1, list2):
    """
    Return the intersection of two lists of dictionaries based on the 'id' key.

    This function filters `list1` and keeps only those dictionaries whose 'id' exists in `list2`.

    Args:
        list1 (list of dict): First list of dictionaries to filter.
        list2 (list of dict): Second list of dictionaries used as reference for 'id' values.

    Returns:
        list of dict: A list of dictionaries from `list1` whose 'id' is present in `list2`.
    """
    ids2 = {item['id'] for item in list2}
    return [item for item in list1 if item['id'] in ids2]

In [ ]:
## Read predictions
dict_preds = pd.read_csv('GPT_results.csv').to_dict('records')

test_set = dataset_['test']
test_set = intersect_by_id(test_set, dict_preds)

list_f1_entity = []
list_recall_entity = []
list_precision_entity = []
list_entity = []
list_id = []
list_syn_entity = []
for test_sample, dict_pred in zip(test_set, dict_preds):

    assert test_sample['id'] == dict_pred['id']

    ### Extractive Format Evaluation ###
    tags = extract_tagged_words(dict_pred['Prediction'])
    extractive_pred = {'id': test_sample['id'], 'preds': tags}

    # Get labels in the evaluation format
    dict_labels = {item['word']: item['subcategory'] for item in test_sample['labels']}

    ## Evaluate performance per entity
    for entity in list_entities:
        ## Filter the entity to be evaluated inside the label
        filtered_dict_labels = {key: value for key, value in dict_labels.items() if value == entity}

        ## Filter the entity to be evaluated that were predicted by the model
        filtered_tags = [sample for sample in extractive_pred['preds'] if sample['subcategory'] == entity]

        filtered_extractive_pred = {'id': test_sample['id'], 'preds': filtered_tags}

        ## Check if the entity exists inside the label to run evaluation
        if len(filtered_dict_labels) > 0:
            f1, recall, precision = eval_entity(filtered_extractive_pred, filtered_dict_labels, verbose=False)

            list_f1_entity.append(f1)
            list_recall_entity.append(recall)
            list_precision_entity.append(precision)
            list_entity.append(entity)
            list_id.append(test_sample['id'])

            # Save if it is synthetic or not
            list_syn_entity.append(test_sample['synthetic'])


### Create a dataframe with the evaluations
entity_save_df = pd.DataFrame()
entity_save_df['id'] = list_id
entity_save_df['Entity'] = list_entity
entity_save_df['Recall'] = list_recall_entity
entity_save_df['Precision'] = list_precision_entity
entity_save_df['F1'] = list_f1_entity
entity_save_df['synthetic'] = list_syn_entity

entity_save_df.to_csv('GPT_entity_results.csv')


In [ ]:
# Saving results per entity
df_grouped_entity = entity_save_df.groupby('Entity')[['Precision', 'Recall', 'F1']].mean().reset_index()
df_grouped_entity.to_csv('GPT_entity_final_results.csv', index=False)
print(df_grouped_entity)

In [ ]:
# Saving results per entity grouped by real and synthetic
df_grouped_syn_entity = entity_save_df.groupby(['synthetic', 'Entity'])[['Precision', 'Recall', 'F1']].mean().reset_index()
df_grouped_syn_entity.to_csv('GPT_entity_final_grouped_results.csv', index=False)
print(df_grouped_syn_entity)

In [ ]:
# Saving F1 scores grouped by real and synthetic samples
df_results_ = pd.read_csv('GPT_results.csv')
df_results_grouped = df_results_.groupby(['synthetic'])[['Precision', 'Recall', 'F1']].mean().reset_index()
df_results_grouped.to_csv('GPT_grouped_results.csv', index= False)